In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration

MODEL_NAME = "facebook/bart-base"  # or "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test_data_1k.json to test_data_1k.json


In [3]:
from google.colab import files
uploaded = files.upload()

Saving eval_data_1k.json to eval_data_1k.json


In [4]:
from google.colab import files
uploaded = files.upload()

Saving train_data_10k.json to train_data_10k.json


In [6]:
import json
from datasets import Dataset

def load_json_lines(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f]

train_data = load_json_lines("train_data_10k.json")
eval_data = load_json_lines("eval_data_1k.json")
test_data = load_json_lines("test_data_1k.json")

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
test_dataset = Dataset.from_list(test_data)

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train samples: 10000
Eval samples: 1000
Test samples: 1000


In [5]:
import json
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch

In [10]:


def escape_string(s):
    replacements = {
        '(': '[LP]', ')': '[RP]', '<': '[LT]', '>': '[GT]',
        '[': '[LSB]', ']': '[RSB]', '{': '[LCB]', '}': '[RCB]',
        '|': '[PIPE]', '&': '[AMP]', '_': '[UND]', ' ': '[SPACE]'
    }
    for k, v in replacements.items():
        s = s.replace(k, v)
    return s

def unescape_string(s):
    replacements = {
        '[LP]': '(', '[RP]': ')', '[LT]': '<', '[GT]': '>',
        '[LSB]': '[', '[RSB]': ']', '[LCB]': '{', '[RCB]': '}',
        '[PIPE]': '|', '[AMP]': '&', '[UND]': '_', '[SPACE]': ' '
    }
    for k, v in replacements.items():
        s = s.replace(k, v)
    return s

def load_and_escape_dataset(json_path):
    data = []
    with open(json_path) as f:
        for line in f:
            entry = json.loads(line)
            data.append({
                'input_text': escape_string(entry['PFD']),
                'target_text': escape_string(entry['PID'])
            })
    return data
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def tokenize(example):
    inputs = tokenizer(example['input_text'], truncation=True, padding='max_length', max_length=256)
    targets = tokenizer(example['target_text'], truncation=True, padding='max_length', max_length=256)
    inputs['labels'] = targets['input_ids']
    return inputs

train_data = load_and_escape_dataset("train_data_10k.json")
eval_data = load_and_escape_dataset("eval_data_1k.json")

train_dataset = Dataset.from_list(train_data).map(tokenize, remove_columns=['input_text', 'target_text'])
eval_dataset = Dataset.from_list(eval_data).map(tokenize, remove_columns=['input_text', 'target_text'])

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

training_args = TrainingArguments(
    output_dir="bart-pfd-pid",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="logs",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)



Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<ipython-input-10-5e3044ca24d6>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhritichandan1 (dhritichandan1-9) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.020500,0.009829
2,0.010500,0.007551
3,0.008900,0.006643


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3750, training_loss=0.023456611887613932, metrics={'train_runtime': 2499.7152, 'train_samples_per_second': 12.001, 'train_steps_per_second': 1.5, 'total_flos': 4573023436800000.0, 'train_loss': 0.023456611887613932, 'epoch': 3.0})

In [17]:
def predict(input_text):
    model.eval()
    inputs = tokenizer(escape_string(input_text), return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=526)

    # Decode and unescape
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    final_outputs = [unescape_string(pred) for pred in decoded]

    return final_outputs[0]  # return the clean, unescaped prediction

In [18]:
print(predict("(raw)(pp)(v)(mix)<1(r)(v)(splt)[(prod)](v)1"))
print(predict("(raw)(hex){1}(mix)<&|(raw)(v)"))

# Evaluate on test dataset
sample_index = 0
sample_pfd = test_dataset[sample_index]["PFD"]
print("Input:", sample_pfd)
print("Prediction:", predict(sample_pfd))
print("True PID:", test_dataset[sample_index]["PID"])

[LSB]LP[RSC]raw[LSb[RSb]RP[RSF][LSB.[RSB)LP}hex[LSPRSB
[LSB]LP[RSC]raw[LSb[RSb]RP[RSD][LSB.][LP[RB]hex[LSP[RSP]RP,[RSB{1}[LSP.RSB)LP[ RSB]C[LSBP[RSF]RP(RSF]{[TCB]TC[RCF]_1]2[LSp[RSBF]LP.[RSC][mix[LSLSB‘RSB’RP[RsB]LT[RSM]&|[LSLSPRSB
Input: (raw)(pp)(v)(v)(mix)<&|(raw)(hex)(v)&|(mix)<&|(raw)(v)&|(pp)(v)(mix)<1(r)(v)(splt)[(prod)](v)1
Prediction: [LSB]LP[RSC]raw[LSb[RSb]RP[RSF][LSB.]{FFC}_1[LSPRSB)LT[RSD][UNB]2[LSD[RSBB]LP(RSB])v[LSLSB�RSB‘RP[ RSB]&|[LSLSRSB].LP[RB]tank[LSC[RS B]RP,][/LSB [RSB ]LSB][RSB,LP[SB]C[LSF[RS
True PID: (raw)(hex)<_1(C){TC}_1(C){FC}_2(v)<_2(mix)<&|(raw)(pp)[(C){M}](C){PI}(C){FC}_3(v)<_3(C){FC}_4(v)&<_4|(mix)<&|(raw)(C){FC}_5(v)&<_5|(pp)[(C){M}](C){PI}(C){FC}_6(v)<_6(mix)<1(r)<_7[(C){TC}_7][(C){LC}_8](v)<_8(splt)[(prod)](C){FC}_9(v)1<_9


In [ ]:
import json
from datasets import Dataset

def load_json_lines(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f]

train_data = load_json_lines("train_data_10k.json")
eval_data = load_json_lines("eval_data_1k.json")
test_data = load_json_lines("test_data_1k.json")

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
test_dataset = Dataset.from_list(test_data)

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train samples: 10000
Eval samples: 1000
Test samples: 1000


In [ ]:
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['PFD']
    targets = examples['PID']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

    # Replace padding token ID in labels with -100
    labels_ids = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label_seq]
        for label_seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./bart_pid_model",
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    save_total_limit=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="tensorboard",
    fp16=torch.cuda.is_available()
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)

<ipython-input-8-fd98f231fc32>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
trainer.save_model('./bart_pid_model')
tokenizer.save_pretrained('./bart_pid_model')

Step,Training Loss,Validation Loss
100,0.862000,0.205338
200,0.221900,0.129072
300,0.145900,0.098834
400,0.119900,0.079490
500,0.098100,0.077349
600,0.084000,0.056554
700,0.077600,0.051816
800,0.069000,0.049344
900,0.066200,0.047124
1000,0.059600,0.038517


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.862000,0.205338
200,0.221900,0.129072
300,0.145900,0.098834
400,0.119900,0.079490
500,0.098100,0.077349
600,0.084000,0.056554
700,0.077600,0.051816
800,0.069000,0.049344
900,0.066200,0.047124
1000,0.059600,0.038517


('./bart_pid_model/tokenizer_config.json',
 './bart_pid_model/special_tokens_map.json',
 './bart_pid_model/vocab.json',
 './bart_pid_model/merges.txt',
 './bart_pid_model/added_tokens.json')

In [ ]:
def pfd_to_pid_bart(pfd_text):
    input_ids = tokenizer.encode(pfd_text, return_tensors="pt", max_length=3000, truncation=True).to(device)
    output_ids = model.generate(input_ids, max_length=2000, num_beams=4, early_stopping=False)
    return tokenizer.decode(output_ids[0], skip_special_tokens=False)

# Example:
sample_index = 0
sample_pfd = test_dataset[sample_index]["PFD"]
print("Input:", sample_pfd)
print("Prediction:", pfd_to_pid_bart(sample_pfd))
print("True PID:", test_dataset[sample_index]["PID"])

Input: (raw)(pp)(v)(v)(mix)<&|(raw)(hex)(v)&|(mix)<&|(raw)(v)&|(pp)(v)(mix)<1(r)(v)(splt)[(prod)](v)1
Prediction: </s><s>(raw)(hex)(C){TC}_1(C){FC}_2(v)&<_2|(mix)<1(r)<_3<&|(raw(pp)[(C)M}](C){PI}(C(){FC}}_3(v]<_4(C {FC}&_5|[(C)[TC}][(C)(LC}_6][{tout}(raw)](v)(splt)[(prod)](CZI}_7<_7(v)]1<_8</s>
True PID: (raw)(hex)<_1(C){TC}_1(C){FC}_2(v)<_2(mix)<&|(raw)(pp)[(C){M}](C){PI}(C){FC}_3(v)<_3(C){FC}_4(v)&<_4|(mix)<&|(raw)(C){FC}_5(v)&<_5|(pp)[(C){M}](C){PI}(C){FC}_6(v)<_6(mix)<1(r)<_7[(C){TC}_7][(C){LC}_8](v)<_8(splt)[(prod)](C){FC}_9(v)1<_9
